In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import operator

f = open('wudi.json',)
wudi = json.load(f)
wudi = {int(k):v for k, v in wudi.items()}


f = open('mudi.json',)
mudi = json.load(f)
mudi = {int(k):v for k, v in mudi.items()}
f.close()

num_users = len(np.unique(np.array(list(wudi.keys()))))
num_materials = len(np.unique(np.array(list(mudi.keys()))))

In [22]:


EMBEDDING_SIZE = 50

class RecommenderNet(keras.Model):
    def __init__(self, num_users, num_materials, embedding_size, **kwargs):
        super(RecommenderNet, self).__init__(**kwargs)
        self.num_users = num_users
        self.num_materials = num_materials
        self.embedding_size = embedding_size
        self.user_embedding = layers.Embedding(
            num_users,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.user_bias = layers.Embedding(num_users, 1)
        self.material_embedding = layers.Embedding(
            num_materials,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.material_bias = layers.Embedding(num_materials, 1)

    def call(self, inputs):
        user_vector = self.user_embedding(inputs[:, 0])
        user_bias = self.user_bias(inputs[:, 0])
        material_vector = self.material_embedding(inputs[:, 1])
        material_bias = self.material_bias(inputs[:, 1])
        dot_user_material = tf.tensordot(user_vector, material_vector, 2)
        # Add all the components (including bias)
        x = dot_user_material + user_bias + material_bias
        # The sigmoid activation forces the rating to between 0 and 1
        return tf.nn.sigmoid(x)

In [29]:
M = RecommenderNet(num_users, num_materials, 50)
M.load_weights("material_recommed")


In [30]:
##### Input #########

one_user_id=29606863  ### existing_wholeasler_id

#################


one_user_wholesaler=[]
for i in range(num_materials):
    one_user_wholesaler.append(one_user_id)
one_user_material=[]
for i in range(num_materials):
    one_user_material.append(i)

one_user=pd.DataFrame({'wholesaler':one_user_wholesaler})
one_user['material']=one_user_material
one_user["wholesaler"] = [wudi[x] for x in one_user["wholesaler"]]

key_list_1 = list(mudi.keys())
val_list_1 = list(mudi.values())

preds=[]
for i in range(len(one_user)):
    preds.append((float(M.predict(np.asarray([one_user.iloc[i]]))[0]), key_list_1[val_list_1.index(one_user['material'][i])])) 

preds
#preds=dict(sorted(preds.items(), key=operator.itemgetter(1), reverse=True)[:10])
#ratings_for_all = pd.DataFrame(preds)
#ratings_for_all



[(0.3127284646034241, 10946),
 (0.32490432262420654, 19898),
 (0.34003204107284546, 80176),
 (0.3273070454597473, 66989),
 (0.3318966031074524, 3337),
 (0.3226690888404846, 48859),
 (0.32548287510871887, 29777),
 (0.3429769277572632, 33335),
 (0.32640349864959717, 5983),
 (0.34272512793540955, 3372),
 (0.34035128355026245, 11574),
 (0.32083073258399963, 20614),
 (0.3200548589229584, 59874),
 (0.32155171036720276, 71317),
 (0.32231295108795166, 63311),
 (0.35291534662246704, 9974),
 (0.3412090837955475, 48537),
 (0.35375288128852844, 37961),
 (0.342352956533432, 59847),
 (0.32911747694015503, 3338),
 (0.31308674812316895, 10947),
 (0.3346834182739258, 6013),
 (0.3317432701587677, 59848),
 (0.3511943221092224, 77313),
 (0.3219517469406128, 76623),
 (0.3446580171585083, 10965),
 (0.3662281036376953, 66839),
 (0.3409616947174072, 10957),
 (0.3585160970687866, 59873),
 (0.34834054112434387, 3410),
 (0.34267953038215637, 55357),
 (0.3475664258003235, 16550),
 (0.37181025743484497, 16519),
 (

In [31]:
preds.sort(reverse=True)
preds

[(0.37819695472717285, 60505),
 (0.37774673104286194, 76626),
 (0.37656357884407043, 10958),
 (0.3737594783306122, 58328),
 (0.37227770686149597, 63478),
 (0.37181025743484497, 16519),
 (0.3715629577636719, 78313),
 (0.3700687885284424, 58941),
 (0.36664295196533203, 53566),
 (0.3662281036376953, 66839),
 (0.36619171500205994, 3411),
 (0.3637864291667938, 72548),
 (0.36252743005752563, 67562),
 (0.3621634244918823, 81411),
 (0.36211663484573364, 39616),
 (0.3620527982711792, 58277),
 (0.3617555499076843, 76828),
 (0.3611491620540619, 81409),
 (0.3606860637664795, 81410),
 (0.36060699820518494, 65716),
 (0.3585160970687866, 59873),
 (0.3582419753074646, 15793),
 (0.3581279516220093, 62474),
 (0.3576759397983551, 59941),
 (0.3571714162826538, 15709),
 (0.355608195066452, 15757),
 (0.3540346026420593, 34906),
 (0.35375288128852844, 37961),
 (0.3531637489795685, 63479),
 (0.35291534662246704, 9974),
 (0.35287928581237793, 59939),
 (0.35286563634872437, 59940),
 (0.35279858112335205, 80417)